# 0. Setup

### Run my mini framework for Plotly visualizations

In [ ]:

%run ./PlotLee_Mini/setup_notebook.ipynb

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict
import json
import ast
from tqdm import tqdm 
import time
import requests
from bs4 import BeautifulSoup, SoupStrainer
import pprint
import logging
import re
# from kaggle_secrets import UserSecretsClient
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import swifter
import itertools
from geopy.geocoders import Nominatim
# from nltk.stem.snowball import SnowballStemmer
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
 
 
# STOPWORDS = set(stopwords.words('english'))
px.defaults.template = 'bnw'

### Load Client

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

### Run Logger

In [ ]:
logger = logging.getLogger(__name__)


In [ ]:
if not logger.hasHandlers():      
    filename = "linkedin-job-data.log"
    ch = logging.FileHandler(filename)
    ch.setLevel(logging.WARNING)
    ## Handler also needs formatter
    formatter = logging.Formatter('%(levelname)s -- %(asctime)s -- %(message)s')
    # add formatter to ch
    ch.setFormatter(formatter)
    # add ch to logger
    logger.addHandler(ch)
    print(f"Added File Handler {filename} to Logger")

### Load and Save Stopwords to GBQ table if not already done

In [ ]:
import requests
stopwords_file = requests.get("https://gist.githubusercontent.com/sebleier/554280/raw/7e0e4a1ce04c2bb7bd41089c9821dbcf6d0c786c/NLTK's%2520list%2520of%2520english%2520stopwords")
STOPWORDS = str(stopwords_file.text).split("\n")

In [ ]:
all_datasets = [dataset.dataset_id for dataset in client.list_datasets()]

In [ ]:
# with open("./useful_code_functions/client_dataset_creation_functions.txt", mode="w") as f:
#     f.write(str(type(client)) + "\n")
#     f.write("\n\t".join([function for function in dir(client) if not function.startswith("_")])) 

In [ ]:
if "helper" in all_datasets:
    print("helper already exists")
else:
    client.create_dataset(dataset="helper")
    pd.DataFrame(STOPWORDS, columns=["stopword"]).to_gbq(f"helper.stopwords")

# 1. Data Exploration & Cleaning

### Load CSVs

In [ ]:
df_linkedin_listing_usa = pd.read_csv("./linkedin-data-analyst-jobs-listings-csv/linkedin-jobs-usa.csv")
df_linkedin_listing_usa["country"] = "USA"

In [ ]:
df_linkedin_listing_canada = pd.read_csv("./linkedin-data-analyst-jobs-listings-csv/linkedin-jobs-canada.csv")
df_linkedin_listing_canada["country"] = "Canada"

In [ ]:
assert list(df_linkedin_listing_usa.columns) == list(df_linkedin_listing_canada.columns), \
"Columns not equal, cannot concat vertically"

In [ ]:
df_linkedin_listing = pd.concat([df_linkedin_listing_usa, df_linkedin_listing_canada])

In [ ]:
df_linkedin_listing.info()

In [ ]:
df_linkedin_listing.head(5)

I noticed that there are some unnormalized JSONs in the criteria column. I will normalize the criteria column and find out the nullity of the normalized criteria column.

## JSON Normalization

Let's normalize the criteria column

In [ ]:
criteria_records = []
for criteria_string_form in df_linkedin_listing["criteria"]:
    criteria_array = None
    try:
        criteria_array = ast.literal_eval(criteria_string_form)
    except Exception as e:
        logger.warn("%s %s", e, criteria_string_form)
        continue    
    criteria_records.append({k:v for criteria_dict in criteria_array for k,v in criteria_dict.items()})

# criteria_df = df_linkedin_listing.iloc[0:3].apply(convert_jsons_to_table, axis=1)
criteria_records[0]

In [ ]:
criteria_df = pd.DataFrame.from_records(criteria_records)

In [ ]:
# Validation Cell
criteria_df.info()

In [ ]:
df = pd.merge(df_linkedin_listing, criteria_df, left_index=True, right_index=True)

In [ ]:
df.head(1)

Looks good! Let's drop the criteria column which holds the json that we have already normalized into other columns

In [ ]:
df.drop("criteria", axis=1, inplace=True)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.info()

## Some Seniority Levels are "Not Applicable"

Let's change those to np.nan

In [ ]:
df["Seniority level"].head()

In [ ]:
df["Seniority level"] = df["Seniority level"].replace("Not Applicable", np.nan)
df["Seniority level"].info()

## Some Job Descriptions are marked as the string 'Nan' instead of being an actual null value. 
Change those 'Nan' strings to np.nan

In [ ]:
df.loc[169, "description"]

In [ ]:
## Replace 'Nan' strings with np.nan
df["description"] = \
    df["description"].replace("^(?i)nan$", np.nan, regex=True)

In [ ]:
df.to_csv("./linkedin-data-analyst-jobs-listings-csv/Linkedin_Jobs_American_and_Canadian.csv")

## Plot Basic Distributions for Each Column

In [ ]:
df.info()   

In [ ]:
title_tallies = df["title"].value_counts().reset_index()

In [ ]:
titles_table = title_tallies.copy()
titles_table["percent"] = ((titles_table["title"] / len(df) * 100).round(0)).astype(str) + "%"
titles_table["plotly_text"] = titles_table["index"] + "<br>" + titles_table["percent"] 
titles_table.loc[3:, "plotly_text"] = ""

In [ ]:
fig1 = px.pie(title_tallies, names="index", values="title", color="index",
             color_discrete_sequence=px.colors.qualitative.Pastel2)
remove_px_attributes(fig1)
fig1.update_traces(texttemplate="%{text}", text=titles_table["plotly_text"])
fig1.add_annotation(text=f"Sample Size: {sum(df['title'].isnull() == False)}", xref="paper", yref="paper", xanchor="right",
                   yanchor="top", x=1, y=-0.1, ax=0, ay=0)
fig1.update_layout(
    uniformtext_minsize=7, uniformtext_mode="hide",
    title=dict(
    text="Types of Jobs in Dataset",
    x=0.05,
    xanchor="left"
), margin=dict(t=100))

In [ ]:
df["posted_date"] = pd.to_datetime(df["posted_date"], format="%Y-%m-%d")

In [ ]:
fig2 = px.line(df["posted_date"].value_counts().sort_index())
fig2.update_yaxes(title="Number of Jobs")
fig2.update_xaxes(title="Posted Date")
fig2.update_layout(title="Number of Jobs Over Time", margin=dict(r=50))
fig2.update_traces(hovertemplate="<b>Date:</b> %{x}<br><b>Number of Jobs:</b> %{y}<br>", mode="lines+markers")
fig2.add_annotation(text=f"Sample Size: {sum(~df['posted_date'].isnull())}", xref="paper", yref="paper", xanchor="right",
                   yanchor="top", x=1, y=-0.1, ax=0, ay=0)
fig2

In [ ]:
fig3 = px.histogram(df, x="onsite_remote")
remove_px_attributes(fig3)
# add_percent_labels_for_hist(fig3, custom=True, custom_text_arr=df["onsite_remote"].value_counts())
add_annotation_for_figure(fig3, f"Sample Size  {sum(df['onsite_remote'].isnull() == False)}", 
                          x_anchor="right", x=1)
fig3.update_yaxes(title=dict(text="count", standoff=20))
fig3.update_xaxes(title="onsite_remote")
fig3.update_traces(text=df["onsite_remote"].value_counts(sort=False))
fig3.update_layout(title="Number of Jobs by Work Style", margin=dict(l=100), coloraxis_showscale=False, bargap=0.6)

In [ ]:
industry_counts = df["Industries"].value_counts().reset_index()
industries_table = industry_counts.copy()
industries_table["percent"] = ((industries_table["Industries"] / len(df[~df["Industries"].isnull()]) * 100).round(0)).astype(str) + "%"
industries_table["plotly_text"] = industries_table["index"] + "<br>" + industries_table["percent"] 
industries_table.loc[5:, "plotly_text"] = ""
industries_table["plotly_text"] = break_text(industries_table["plotly_text"].tolist())

In [ ]:
fig4 = px.pie(industry_counts, names="index", values="Industries",
              color_discrete_sequence=px.colors.qualitative.Set3)
remove_px_attributes(fig4)
add_annotation_for_figure(fig4, f"Sample Size:  {sum(df['Industries'].isnull() == False)}",
                          x_anchor="right", x=1)
fig4.update_layout(
    uniformtext_minsize=12, uniformtext_mode="show", title=dict(
    text="Jobs by Sector",
    xanchor="left",
    x=0.05,
    y=0.95,
    ), margin=dict(t=120)
)
fig4.update_traces(texttemplate="%{text}", text=industries_table["plotly_text"])
fig4

# for i, trace in enumerate(fig4.data):
#     print(trace["name"])
#     if not trace["name"] in items_to_show:
#         fig4.data[i]["showlegend"] = False

In [ ]:
fig5 = px.pie(df["Job function"].value_counts().reset_index(), names="index", values="Job function",
              color_discrete_sequence=px.colors.qualitative.Set3)
remove_px_attributes(fig5)
add_annotation_for_figure(fig5, f"Sample Size:  {sum(df['Job function'].isnull() == False)}",
                          x_anchor="right", x=1)
fig5.update_layout(
    uniformtext_minsize=12, uniformtext_mode="hide", title=dict(
    text="Jobs by Function",
    xanchor="left",
    x=0.05,
    y=0.95)
)
fig5.update_traces(textposition="inside", texttemplate="%{percent}")
fig5

# for i, trace in enumerate(fig4.data):
#     print(trace["name"])
#     if not trace["name"] in items_to_show:
#         fig4.data[i]["showlegend"] = False

In [ ]:
# employment_counts = df["Employment type"].value_counts(sort=False).reset_index()

In [ ]:
# employment_counts["percentage_decrease"] = np.where(
#     np.isnan(employment_counts["Employment type"].shift(-1)),
#     "",
#     (
#         (employment_counts["Employment type"].shift(-1) - employment_counts["Employment type"]) / employment_counts["Employment type"] * 100
#     ).round(2).astype(str) + "%"
# )

In [ ]:
# employment_counts

In [ ]:
fig6 = px.histogram(df[~df["Employment type"].isnull()], 
                    x="Employment type")
remove_px_attributes(fig6)
# add_percent_labels_for_hist(fig6, custom=True, custom_text_arr=df["Employment type"].value_counts().tolist())
add_annotation_for_figure(fig6, f"Jobs by Employment Type:  {sum(df['Employment type'].isnull() == False)}", 
                          x_anchor="right", x=1)
fig6.update_traces(text=df["Employment type"].value_counts(sort=False),
                   marker_color=px.colors.qualitative.D3[1])
fig6.update_yaxes(title=dict(text="count", standoff=20))
fig6.update_xaxes(title="Employment type")
fig6.update_layout(title="Number of Jobs by Employment Type", margin=dict(l=100), coloraxis_showscale=False,
                   bargap=0.4)

# fig6.add_trace(go.Scatter(x=employment_counts["index"], y=employment_counts["Employment type"],
#                           mode="lines+markers"))

# for rowNum, row in employment_counts.iterrows():
#     if rowNum == len(employment_counts) - 1:
#         continue

#     y1 = row["Employment type"]
#     y2 = employment_counts.loc[rowNum + 1, "Employment type"]
#     fig6.add_annotation(x=row["index"], y=y1 - (y1 - y2) / 2, text=row["percentage_decrease"], xshift=150, ax=0, ay=0)
# fig6

In [ ]:
df.loc[~df["salary"].isnull(), "salary"]


Salary data looks unclean so let's fix that

In [ ]:
df_salary = pd.DataFrame()
df_salary["salary_has_CA"] = df["salary"].replace("[\r\n\s,]+", "", regex=True).str.contains("CA")
df_salary["salary_text"] = (
    df["salary"].replace("[\r\n\s,]+", "", regex=True)
    .replace("CA", "", regex=True) # replace spaces and unexpected text
)

salaries = df_salary["salary_text"].str.split("-")

## Left bound and upper bound salaries
df_salary["salary_lb"] = salaries.str[0].str.strip("$").astype(float)
df_salary["salary_ub"] = salaries.str[1].str.strip("$").astype(float)

In [ ]:
df_salary.describe()

In [ ]:
fig = px.histogram(df_salary, x="salary_lb")
fig.update_layout(title="Distribution of Salary (Lower Bound Distribution)")
add_annotation_for_figure(fig, f"Sample Size: {sum(~df_salary['salary_lb'].isnull())}")
fig


We can see the salary values are bimodal, perhaps even trimodal. Some salaries are written as hourly salaries, while some appear to be monthly and other salaries appear to be annual.

Let's **break down** the salary ranges

In [ ]:
df_salary_hourly = df_salary.loc[df_salary["salary_lb"] <= 2000] 

In [ ]:
df_salary_hourly.describe()

In [ ]:
df_salary.loc[(df_salary["salary_lb"] >= 3000) & (df_salary["salary_lb"] <= 7000)] 

I think it's safe the say that the salary date being provided is multimodal. Let's convert these salaries to annual salaries

In [ ]:
previous_less_than_2000 = (df_salary["salary_lb"] <= 2000)

In [ ]:
df_salary.loc[df_salary["salary_lb"] <= 2000, ["salary_lb", "salary_ub"]]  = df_salary.loc[df_salary["salary_lb"] <= 2000, ["salary_lb", "salary_ub"]] * 40 * 4 * 12  

In [ ]:
df_salary.loc[(df_salary["salary_lb"] >= 3000) & (df_salary["salary_lb"] <= 7000) & ~previous_less_than_2000, ["salary_lb", "salary_ub"]] = \
df_salary.loc[(df_salary["salary_lb"] >= 3000) & (df_salary["salary_lb"] <= 7000) & ~previous_less_than_2000, ["salary_lb", "salary_ub"]] * 12

In [ ]:
df_salary["salary_text"] = np.where(
    df_salary["salary_lb"].isnull() & df_salary["salary_ub"].isnull(), 
    np.nan, 
    "$" + df_salary["salary_lb"].astype(str) + "-" + df_salary["salary_ub"].astype(str) 
)

Check distribution of salaries again

In [ ]:
fig = px.histogram(df_salary, x="salary_lb")
fig.update_layout(title="Distribution of Salary (Lower Bound Distribution)")
add_annotation_for_figure(fig, f"Sample Size: {sum(~df_salary['salary_lb'].isnull())}")
fig


In [ ]:
fig = px.histogram(df_salary, x="salary_ub")
fig.update_layout(title="Distribution of Salary (Upper Bound Distribution)")
add_annotation_for_figure(fig, f"Sample Size: {sum(~df_salary['salary_ub'].isnull())}")
fig


Much better!

In [ ]:
df_salary_analysis = df.merge(df_salary, left_index=True, right_index=True)

Apply a conversion rate on salary values

In [ ]:
# Conversion Rate
df_salary_analysis["salary_ub"] = np.where(df_salary_analysis["country"] == "USA", df_salary_analysis["salary_ub"] * 1.33, df_salary_analysis["salary_ub"])
df_salary_analysis["salary_lb"] = np.where(df_salary_analysis["country"] == "USA", df_salary_analysis["salary_lb"] * 1.33, df_salary_analysis["salary_lb"])

Convert some ordinal values from strings to integers to analyze correlations

In [ ]:
df_salary_analysis["Seniority level"].unique()

In [ ]:
df_salary_analysis["Employment type"].unique()

In [ ]:
df_salary_analysis.groupby("Seniority level").agg({"salary_ub": "mean"})

In [ ]:
encode_df = df_salary_analysis.copy()

In [ ]:
encode_df["Seniority level"].replace({"Entry level" : 1, "Associate" : 2, "Mid-Senior level" : 3, "Executive" : 4}, inplace=True)
encode_df["Employment type"].replace({"Volunteer" : 1, "Temporary" : 2, "Contract" : 3, "Full-time" : 4}, inplace=True)

In [ ]:
fig = px.imshow(encode_df.corr(), color_continuous_scale="BuGn")
fig.update_layout(title=dict(text="Correlations Between Ordinal Job Attributes", x=0.05, y=0.96, xanchor="left", font_size=24),
                  margin=dict(b=160))

**Findings:**
- Salary has a somewhat strong correlation with seniority level. This is expected

In [ ]:
fig7 = px.box(df_salary_analysis, x="salary_lb", color="Seniority level")
remove_px_attributes(fig7)
add_annotation_for_figure(fig7, f"Sample Size: {min(sum(~df_salary_analysis['salary_lb'].isnull()), sum(~df_salary_analysis['Seniority level'].isnull()))}",
                          x_anchor="center", x=1.1)
fig7.update_layout(showlegend=True, title="Salary (Lower Bound) Based on Seniority Level (in CAD)")
fig7.update_xaxes(showgrid=True)
fig7.update_traces(hoverinfo="x")

In [ ]:
fig8 = px.histogram(df_salary_analysis, x="Seniority level", color="Seniority level")
remove_px_attributes(fig8)
add_percent_labels_for_hist(fig8, custom=True, custom_text_arr=df_salary_analysis["Seniority level"].value_counts(sort=False))
add_annotation_for_figure(fig8, f"Sample Size: {min(sum(~df_salary_analysis['salary_lb'].isnull()), sum(~df_salary_analysis['Seniority level'].isnull()))}",
                          x_anchor="center", x=1)
fig8.update_layout(showlegend=True, title="Jobs by Seniority Level", bargap=0.5)
fig8.update_xaxes(showgrid=True)
fig8.update_traces(hoverinfo="x")

## Write Out Jobs Tables to Bigquery

In [ ]:
df_salary_analysis = df_salary_analysis.loc[:, df_salary_analysis.columns[~df_salary_analysis.columns.isin(["link", "salary"])]]

In [ ]:
df_salary_analysis = \
(
    df_salary_analysis.reset_index()
                      .rename(columns={
                          "Seniority level" : "seniority_level", 
                          "Employment type" : "employment_type", 
                          "Job function": "job_function", 
                          "Industries" : "industries",
                          "index" : "job_id"
                       })
)

In [ ]:
df_salary_analysis.head()

In [ ]:
df_salary_analysis.info()

### Load DATASET_NAME and TABLE_NAME from .env variables

In [ ]:
import os
from dotenv import load_dotenv

_ = load_dotenv()
DATASET_NAME = os.getenv('DATASET_NAME')
TABLE_NAME = os.getenv('TABLE_NAME')

### Write out table to GBQ

In [ ]:
df_salary_analysis.to_gbq(f"{DATASET_NAME}.{TABLE_NAME}", if_exists="replace")

## Make Our Linkedin Jobs Exploration Dashboard

To extract this report, open up a jupyter notebook server and save the report cell as an **Embed**

In [ ]:
from ipywidgets import HBox, VBox
figures = [fig1, fig4, fig5, fig2, fig3, fig6, fig7]
jup_figures = convert_figures_to_figurewidgets(figures)
pie_charts = VBox([jup_figures[0], jup_figures[1], jup_figures[2]])
bar_charts = VBox([jup_figures[4], jup_figures[5]])
time_chart = jup_figures[3]
salary_chart_by_seniority = jup_figures[6]
report = VBox([pie_charts, bar_charts, time_chart, salary_chart_by_seniority], layout=dict(height="3500px", margin="0px 0px 0px 0px", padding="0px 0px 0px 0px",
                                                                                           justify_content="space-between", background="#f0f0f0"))

In [ ]:
report  

## Wandb Exploration

Let's use Wandb to explore individual data rows easier

Credit to https://www.kaggle.com/code/ayuraj/interactive-eda-using-w-b-tables/notebook for code and inspiration

In [ ]:
# # Import wandb
# import wandb

# try:
#     secret_value_0 = os.getenv("WANDB_API")
#     wandb.login(key=secret_value_0)
    
#     anony=None
# except Exception as e:
#     anony = "must"
#     print('If you want to use your W&B account, go to Add-ons -> Secrets and add your W&B access token. Use the Label name as "wandb_api". \nGet your W&B access token from here: https://wandb.ai/authorize')

In [ ]:
# run = wandb.init(project='eda', anonymous=None) # W&B Code 1

In [ ]:
# # Initialize a W&B run to log images
# data_at = wandb.Table(columns=df_salary_analysis.columns.tolist()) # W&B Code 2
# for i in tqdm(range(len(df_salary_analysis))):
#     row = df_salary_analysis.loc[i]
#     data_at.add_data(*tuple(row.values[0:])) # W&B Code 3

# wandb.log({'LinkedIn Job Data': data_at}) # W&B Code 4
# wandb.finish() # W&B Code 5

# 3. Extract Entities From Job Description

There are a few entities I want to extract from job descriptions: 
- **Programming Languages**: I want to see what programming languages companies are interested in
- **Soft Skills**:  I want to see what soft skills are expected of Data Analyst Jobs
- **Hard Skills**: I want to understand what hard skills are expected of Data Analyst Jobs

## 3.1 Use Spacy's Pretrained LLM Model to Extract Entitiess

In [ ]:
# !pip freeze | grep spacy

In [ ]:
!python -m spacy download en_core_web_sm 


In [ ]:
# wordsegment is a library that probabilistically separates an undelimited piece of text. For example, it turns 'hiimjohn' into 'hi im john'
# We may either use/not use wordsegment for entity extraction
from wordsegment import load as load_words, segment, clean as segment_clean, WORDS, BIGRAMS, UNIGRAMS
import string

In [ ]:
load_words()
len(BIGRAMS), len(UNIGRAMS), len(WORDS)

In [ ]:
from nltk.corpus import words
setofwords = set(words.words())

In [ ]:
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

### Example Entity Extraction

In [ ]:
job_description_example = nlp(df.loc[500, "description"])

In [ ]:
displacy.serve(job_description_example, style="ent")

## Entity Extraction

In [ ]:
def find_all_named_entities(row, test=False, segment_sentence=False):
    '''
        Assumption: Segment every word
    '''
    job_description = row["description"] if test is False else row

    # Skip np.nan
    if isinstance(job_description, float):
        return []
        
    nlp_tokens = nlp(job_description)
    all_entities = [[token.text, token.label_] for token in nlp_tokens.ents 
                    if token.text.lower() not in STOPWORDS]
    
    final_entities = []
    if segment_sentence is True:
        # Load 
        for ent_text, ent_label in all_entities:
            try:
                    is_entity_found = False
                    texts = segment(ent_text) 
                    for text in texts:
                        if text.lower() in UNIGRAMS and \
                            text.lower() in ENTITIES:
                            final_entities.append(text)
                            is_entity_found = True

                    if is_entity_found is False:
                        final_entities.append(ent_text)
            except ValueError as e:
                logger.warn("Unexpected text segmentation error: ", e)
                final_entities.append(ent_text)
    else:
        final_entities = all_entities
        
    # np_final_word_array = np.expand_dims(np.array(all_words), axis=1)
    # np_job_description_index = np.expand_dims([row.name if test is False else 0] 
    #                                           * len(all_words), axis=1)
    # return np.concatenate((np_job_description_index, np_final_word_array), axis=1).tolist()
    return final_entities

all_entities = df.swifter.apply(find_all_named_entities, axis=1).explode()

In [ ]:
all_entities.info()

In [ ]:
# Preprocess List of Entities derived from Spacy LLM
entity_frame = pd.DataFrame(all_entities)
entity_frame["entity"] = entity_frame[0].str[0]
entity_frame["type"] = entity_frame[0].str[1]
entity_frame.drop(columns=[0], inplace=True)
entity_frame.head()

In [ ]:
entity_frame["type"].unique()

In [ ]:
store_value_counts = []
for entity_type in entity_frame["type"].unique():
    store_value_counts.append(
        entity_frame[entity_frame["type"] == entity_type].value_counts()
    )

In [ ]:
len(store_value_counts)

In [ ]:
store_value_counts[0]

In [ ]:
store_value_counts[3]

We can see that clearly spacy's LLM is not trained to identify Data Analyst skills. However we know that utilizing GPT-NeoX or any Generative Autoregressive Model to extract entities with few-shot learning is not feasible for a personal project, as they cost way too much for reasonable entity extraction across many tokens. So I will settle with using what I am given: Spacy's Pre-trained LLMs and use their pre-defined entities to extract insight

In [ ]:
final_frame = entity_frame[entity_frame["type"].isin([
    "ORG", "LOC", "PRODUCT", "PERSON", "GPE", "LAW", "LANGUAGE"
])].reset_index().rename(columns={"index" : "job_id"})

### 3.1.2 Write Out our Entities to GBQ

In [ ]:
final_frame.to_gbq(f"{DATASET_NAME}.job_entities", if_exists="replace")

## 3.2 Try out nlpcloud models for Automatic Entity Extraction based on human input

In [ ]:
import nlpcloud
import time

### 3.2.1 Try llama

In [ ]:
llama = nlpcloud.Client("finetuned-llama-2-70b", "fbc5fd9ec4e8d9a0f42bb8e49edf7e84e2c639fe",
                        gpu=True, lang='en')

In [ ]:
CHUNK_SIZE = 256

In [ ]:
all_entities = []

In [ ]:
split_words = df_salary_analysis.loc[0, "description"].split(" ")

In [ ]:
# for i in range(0, len(split_words), CHUNK_SIZE):
#     # Stay under llama's 256 token limit
#     string_to_get_entities = " ".join(split_words[i:i+CHUNK_SIZE])
#     entity_dict = llama.entities(string_to_get_entities, searched_entity="hard skill")
#     all_entities.extend(entity_dict["entities"])
#     time.sleep(10)


Rate limiting is a problem for us. I do not wish to upgrade my free plan, so either I live with entity extraction under the free plan or I ditch the idea to use these powerful generative autoregressive LLMs. 

In [ ]:
llama.entities("I want to use Tableau and be good at Python", searched_entity="hard skill")

# 4. Extract Words From Job Description

In [ ]:
# tokens = nlp("I am singing in the garden")
# print([token.lemma_.lower() for token in tokens])

In [ ]:
def find_all_words_based_on_spacy_delim(row, test=False, segment_sentence=False):
    '''
        Assumption: Segment every word
    '''
    job_description = row["description"] if test is False else row
    if isinstance(job_description, float):
         return []
    
    # Get rid of dangling punctuations. Keep periods, commas, and semicolons. 
    # Unfortunately can't keep track of newline separators because of poor web scraping practices 
    all_words = re.split(r"[!\"'#$%&'*+/:<=>?@^_`|~()[\]{}\s]+", job_description)
    final_words_array = []

    if segment_sentence is False:
        for word in all_words:
            # If word's punctuation has position, then split. 
            # Assumption is that these punctuations exist at end of sentences.
            if re.search("[,;.]+", word):
                # print(word)
                start = re.search("[,;.]+", word).start(0)
                final_words_array.append(word[:start])
                final_words_array.append(word[start:start+1])
            else:
                final_words_array.append(word)
            # if not, then just add word to array
    else:      
        for word in all_words:
                try:
                    words = segment(word) ## ValueError can occur due to unavailable segmentation
                    final_words_array.extend(words)
                except ValueError:
                    final_words_array.append(word)
    
    return final_words_array
        

all_words = df.swifter.apply(find_all_words_based_on_spacy_delim, axis=1).explode()

In [ ]:
df.loc[0, "description"]

In [ ]:
# with open("useful_code_functions/named_entity_methods.txt", mode="w") as f:
#     f.write(str(type(a)))
#     f.write("\n")
#     f.write(str(type(a.ents[0])))
#     f.write("\n".join(
#         ["\t" + elem for elem in dir(a.ents[0]) if not elem.startswith("_")]
#         )
#     )

Takes around 10 minutes to run machine learning model on ~5000 description items

In [ ]:
# We actually decided to keep punctuations for easier bigram and trigram analysis
# import string
# all_words_no_puncutation = all_words[~all_words.str.match(f"^[{re.escape(string.punctuation)}\s]+$")]

In [ ]:
string.punctuation

In [ ]:
final_words = all_words.reset_index()
final_words.columns = ["job_id", "word"]

In [ ]:
df.loc[0, "description"]

In [ ]:
final_words["word"] = final_words["word"].str.lower()

In [ ]:
final_words = final_words[final_words["word"] != '']

In [ ]:
final_words["row_num"] = range(len(final_words))

In [ ]:
final_words.to_gbq("static_job_warehouse.job_words", if_exists="replace")

In [ ]:
# words_based_on_delim = pd.DataFrame({"Label_ID": all_words.str[0], 
#                                     "Word" : all_words.str[1]})

In [ ]:
# words_based_on_delim.head(10)

In [ ]:
# job_id_1843 = final_words[final_words["job_id"] == 1843]

In [ ]:
# len(job_id_1843)

In [ ]:
# job_id_1843.iloc[400:440]

## Archive

### Performance Testing between pd.Series.apply and pd.DataFrame.apply

In [ ]:
%%timeit
def get_keywords(row):
    job_desc = row["description"].lower()
    has_qualification = job_desc.find("qualification")
    has_requirement = job_desc.find("requirement")
    has_basic_requirement = job_desc.find("basic requirement")
    has_responsibility = job_desc.find("responsibility")
    return [has_qualification, has_requirement, has_basic_requirement, has_responsibility]

new_data = df.apply(get_keywords, result_type="expand", axis=1)

In [ ]:
new_data = df.apply(get_keywords, result_type="expand", axis=1)

In [ ]:
new_data.head(5)

In [ ]:
def get_keywords_series(job_desc):
    job_desc = job_desc.lower()
    has_qualification = job_desc.find("qualification")
    has_requirement = job_desc.find("requirement")
    has_basic_requirement = job_desc.find("basic requirement")
    has_responsibility = job_desc.find("responsibilit")
    return [has_qualification, has_requirement, has_basic_requirement, has_responsibility]



In [ ]:
%%timeit
series = df["description"].apply(get_keywords_series)
new_data_series = pd.DataFrame(series.tolist())

In [ ]:
series = df["description"].apply(get_keywords_series)
new_data_series = pd.DataFrame(series.tolist(), columns=["has_qualification", "has_requirement",
                                                        "has_basic_requirement", "has_responsibility"])

In [ ]:
new_data_series.head(5)

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
 
word_tokens = word_tokenize(example_sent)
# converts the words in word_tokens to lower case and then checks whether
# they are present in stop_words or not
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
#with no lower case conversion
filtered_sentence = []
 
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
 
print(word_tokens)
print(filtered_sentence)

### Performance Test End
